In [6]:
import stim
from dem_builder_standard_LDU import DEM_builder
import os
import numpy as np
import scipy
from surface_code_standard_LDU import Rotated_Surface_Code_standard_LDU
import pymatching
import multiprocessing
from multiprocessing import Pool

In [7]:
#functions

#build DEM associated to lost of ancilla qubits
def build_multi_dem_ancilla(distance,loss):
    loss_round=round(loss,5)
    lattice=Rotated_Surface_Code_standard_LDU(distance,distance,distance,'|0>','Z',measurement_order='z_first',loss_rate=loss_round)
    dem_build=DEM_builder(lattice,no_observable=True)
    path_folder='data/dem/dem_standard_LDU/'
    path='DEM_surface_code_({},{},{})_loss_rate={}'.format(lattice.width,lattice.height,lattice.rounds,lattice.loss_rate)
        
    os.makedirs(path_folder, exist_ok=True) 
    dem_build.build_dem_lost_ancilla_qubits(path_folder+path)

#wrapper to one argument (needed for parallelization)
def multi_args_wrapper_dem_ancilla(args):
   build_multi_dem_ancilla(*args)

#build DEM associated to lost of data qubits
def build_multi_dem_data(distance,loss,strength):
    strength=round(strength,5)
    loss=round(loss,5)
    lattice=Rotated_Surface_Code_standard_LDU(distance,distance,distance,'|0>','Z',measurement_order='z_first',after_CZ=('DEPOLARIZE2',strength),loss_rate=loss)
    dem_build=DEM_builder(lattice,no_observable=True)
    path_folder='data/dem/dem_standard_LDU/'
    path='DEM_surface_code_({},{},{})_loss_rate={}_depo_noise={}'.format(lattice.width,lattice.height,lattice.rounds,lattice.loss_rate,strength)
    
    os.makedirs(path_folder, exist_ok=True) 
    dem_build.build_dem_lost_data_qubits(path_folder+path)

#wrapper to one argument (needed for parallelization)
def multi_args_wrapper_dem_data(args):
   build_multi_dem_data(*args)
    

In [ ]:
#build DEM associated to loss of ancilla qubits in parallel (does not depend on the loss probability)
distance_list=[3,5,7,9,11]
loss_rate=np.linspace(0.001,0.035,35)#list of loss probabilities
nb_core =10 #number of cores used
for L in distance_list:
    with Pool(processes=nb_core) as pool:
        pool.imap(multi_args_wrapper_dem_ancilla,[(L,loss) for loss in loss_rate])
        pool.close()
        pool.join()
            

In [8]:
#build DEM associated to loss of data qubits in parallel (for vanishing loss probability)
distance_list=[3,5,7,9,11]
loss_rate=np.linspace(0.001,0.035,35)#list of loss probabilities
nb_core =10 #number of cores used

for L in distance_list:
    with Pool(processes=nb_core) as pool:
        pool.imap(multi_args_wrapper_dem_data,[(L,loss,0) for loss in loss_rate])
        pool.close()
        pool.join()
            

In [32]:
#build DEM associated to loss of data qubits in parallel (for finite loss probabilities)
distance_list=[3,5,7,9,11]
loss_rate=[0.001,0.002,0.003,0.005,0.007,0.01,0.015,0.02,0.025,0.027]#list of loss probabilities
depo=np.linspace(0.001,0.016,16)#list of depolarizing noise probabilities
nb_core =10 #number of cores used

for L in distance_list:
    for loss in loss_rate:
        with Pool(processes=nb_core) as pool:
            pool.imap(multi_args_wrapper_dem_data,[(L,loss,strength) for strength in depo])
            pool.close()
            pool.join()
            